1) Переделать с привязкой к метке лейбла!!!!!!

In [1]:
import torch
import pandas as pd
import csv
import sys
import os
import numpy as np
import matplotlib.pyplot as plt


### paths

In [2]:
pathaudio = 'Data/train/audio/'
patheyes = 'Data/train/eyes/'
pathface = 'Data/train/face_nn/'
pathkinect = 'Data/train/kinect/'
pathlabels = 'Data/train/labels/'

In [3]:
list_audio = os.listdir(pathaudio)
list_eyes = os.listdir(patheyes)
list_face = os.listdir(pathface)
list_kinect = os.listdir(pathkinect)
list_labels = os.listdir(pathlabels)

In [4]:
print('Количество файлов по папкам:\n1) audio: {}'
      '\n2) eyes: {} \n3) face: {} \n4) kinect: {} \n5) labels: {}'
     .format(len(list_audio), len(list_eyes), len(list_face), len(list_kinect), len(list_labels)))

Количество файлов по папкам:
1) audio: 272
2) eyes: 306 
3) face: 256 
4) kinect: 295 
5) labels: 312


In [5]:
dict_of_labels = {}
count_dict = {}

for label in list_labels:
    count_audio = list_audio.count(label)
    count_eyes = list_eyes.count(label)
    count_face = list_face.count(label)
    count_kinect = list_kinect.count(label)
    
    if count_audio + count_eyes + count_face + count_kinect == 0:
        continue
    
    object_dict = {'audio' : count_audio, 'eyes' : count_eyes, 'face' : count_face, 'kinect' : count_kinect}
    
    dict_of_labels[label] = object_dict
    count_dict[label] = count_audio + count_eyes + count_face + count_kinect

In [6]:
print(len(dict_of_labels))

list(dict_of_labels.items())[:10]

306


[('id5181a851.csv', {'audio': 1, 'eyes': 1, 'face': 1, 'kinect': 1}),
 ('id13feb714.csv', {'audio': 1, 'eyes': 1, 'face': 1, 'kinect': 1}),
 ('idde0a7733.csv', {'audio': 0, 'eyes': 1, 'face': 0, 'kinect': 1}),
 ('id42d71a97.csv', {'audio': 0, 'eyes': 1, 'face': 1, 'kinect': 1}),
 ('id4d4cd53e.csv', {'audio': 1, 'eyes': 1, 'face': 0, 'kinect': 1}),
 ('id571e2b00.csv', {'audio': 0, 'eyes': 1, 'face': 1, 'kinect': 0}),
 ('id911ae0ab.csv', {'audio': 1, 'eyes': 1, 'face': 1, 'kinect': 1}),
 ('id8036ccb4.csv', {'audio': 1, 'eyes': 1, 'face': 1, 'kinect': 1}),
 ('idb9d6496b.csv', {'audio': 0, 'eyes': 1, 'face': 1, 'kinect': 0}),
 ('idd4bd6c5d.csv', {'audio': 1, 'eyes': 1, 'face': 1, 'kinect': 1})]

### 123

In [7]:
set_precision = '{:.' + str(precision) + 'f}'
    try:
        strt[set_precision.format(modal[0, 0] - label[0, 0])] += 1
    except:
        strt[set_precision.format(modal[0, 0] - label[0, 0])] = 1

    try:
        shp[set_precision.format(len(label) / len(modal))] += 1
    except:
        shp[set_precision.format(len(label) / len(modal))] = 1        

    try:
        step[set_precision.format(modal[1, 0] - modal[0, 0])] += 1 
    except:
        step[set_precision.format(modal[1, 0] - modal[0, 0])] = 1

IndentationError: unexpected indent (<ipython-input-7-3b6543375066>, line 2)

In [70]:
def restoration(data, shape, label, precision = 2):
    
    
    set_precision = '{:.' + str(precision) + 'f}'
    
    start = float(set_precision.format(data[0, 0] - label[0, 0]))
    step = float(set_precision.format(data[1, 0] - data[0, 0]))
    #print(start)
    
    labelstep = 0.01
    #print(float(start), float(step))
    #shape = 0
    new_data = []
    
    if start < 0.07:
        for i in range(int(start/labelstep) + 1):
            new_data.append(data[0])      
    else:
        prev_data = np.zeros((int(start / labelstep), shape[1] + 1))
        new_data.append(data[0])
            
            
            
    # Начала совпадают!!
    koef = int(step / labelstep)
#     if koef == 7:
#         pass # отдельно для кинекта (на 20 семплов тега - 3 семпла кинект)
    
    
    
#     else: # 2 and 4
    
    if koef == 7:
        flag = 0
    else:
        flag = -1000000
    
    for i in range(1, len(data)):
        loopstep = float(set_precision.format(data[i][0] - data[i-1][0]))
        if loopstep == step:
            
            
            
            for i in range(koef):
                new_data.append(data[i])
                
            flag += 1
            if flag == 2:
                new_data.pop()
                flag = 0


            pass # Добавляем данных согласно коэффициенту!

        else:
            print('\nattention!!!! HOLE IN DATA!!!!', shape, data.shape, loopstep)
            print()
            add_zeros = np.zeros((int(loopstep / step), shape[1] + 1))
            new_data.append(add_zeros)
                

    try:
        new_data = np.vstack(new_data)
    except ValueError:
        for d in new_data:
            pass
            #print(d.shape)
    
    # final tune
    if shape[0] - new_data.shape[0] < 9:
        # add last elem
        for i in range(shape[0] - new_data.shape[0]):
            new_data = np.vstack([new_data, data[-1]])                       

    elif shape[0] - new_data.shape[0] > 8:
        print('last added zeros!!!', shape[0] - new_data.shape[0])
        new_data = np.vstack([new_data, np.zeros((shape[0] - new_data.shape[0], shape[1] + 1))])   
    
    # if shape / data.shape * (0.02/0.04 for step) == 2/4 => non-separate - add mean values (найти разницу в старте и дополнить старт...)
    # elif shape != and if start == start => step-by-step add (if step[i] > step => add zeros) if под конец shapes != => add zeros to the end
    
    return new_data

In [49]:
def load_lbl(lbl, path, file, labeldata, shape):
    
    if lbl == 1:
        datas = np.array(pd.read_csv(path + file, skiprows=[0], header=None))
        
        # data restore
        restore_data = restoration(datas, shape, labeldata)
        #print(strt)
        print('restore', restore_data.shape)
        
        # shape_assert and delete time-row
        
        
    else:
        restore_data = np.zeros(shape)
        print('zeros data', restore_data.shape)
        
    return restore_data

In [64]:
def data_vizualization(file, audio, eyes, face, kinect, dictt=None, verbose=True):
    
    label_data = np.array(pd.read_csv(pathlabels + file, skiprows=[0], header=None))
    print(label_data.shape)
    
    audio_data = load_lbl(audio, pathaudio, file, label_data, (len(label_data), 36))
    eyes_data = load_lbl(eyes, patheyes, file, label_data, (len(label_data), 6))
    face_data = load_lbl(face, pathface, file, label_data, (len(label_data), 100))
    #print(st)
    kinect_data = load_lbl(kinect, pathkinect, file, label_data, (len(label_data), 27)) # 0.07

    #print(kinect_data.shape)

In [71]:
for key, value in list(dict_of_labels.items())[:100]:
    print('\n', key)
    data_vizualization(key, verbose=False, **value)


 id5181a851.csv
(3082, 8)
restore (3082, 37)
restore (3082, 7)
restore (3082, 101)
last added zeros!!! 84
restore (3082, 28)

 id13feb714.csv
(4114, 8)
restore (4114, 37)
restore (4114, 7)
restore (4114, 101)
last added zeros!!! 109
restore (4114, 28)

 idde0a7733.csv
(5170, 8)
zeros data (5170, 36)
restore (5170, 7)
zeros data (5170, 100)
last added zeros!!! 131
restore (5170, 28)

 id42d71a97.csv
(1973, 8)
zeros data (1973, 36)
restore (1973, 7)
restore (1973, 101)
last added zeros!!! 51
restore (1973, 28)

 id4d4cd53e.csv
(5399, 8)
restore (5399, 37)
restore (5399, 7)
zeros data (5399, 100)
last added zeros!!! 140
restore (5399, 28)

 id571e2b00.csv
(5127, 8)
zeros data (5127, 36)
restore (5127, 7)
restore (5127, 101)
zeros data (5127, 27)

 id911ae0ab.csv
(3934, 8)
restore (3934, 37)
restore (3934, 7)
restore (3934, 101)
last added zeros!!! 98
restore (3934, 28)

 id8036ccb4.csv
(5069, 8)
restore (5069, 37)
restore (5069, 7)

attention!!!! HOLE IN DATA!!!! (5069, 100) (2532, 101) 

In [81]:
0.0666666667*3

0.20000000010000002